In [1]:
import os
import json
import synapseclient
import StringIO
import csv
import pandas
import requests
import boto3
import nda_aws_token_generator
import logging

logger = logging.getLogger("main")
logger.setLevel(logging.DEBUG)
#create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

syn = synapseclient.login(silent=True)

REFERENCE_GUID = 'NDAR_INVRT663MBL'

# This is an old genomics subject
EXCLUDE_GENOMICS_SUBJECTS = ('92027', )
# EXCLUDE_EXPERIMENTS = ('534', '535')

EXCLUDE_EXPERIMENTS = ()

content_type_dict = {'.gz': 'application/x-gzip', '.bam': 'application/octet-stream', 
                     '.zip': 'application/zip'}

metadata_columns = ['src_subject_id', 'experiment_id', 'subjectkey', 'sample_id_original', 
                    'sample_id_biorepository',
                    'subject_sample_id_original', 'biorepository', 'subject_biorepository', 'sample_description',
                    'species', 'site', 'sex', 'sample_amount', 'phenotype', 'comments_misc']

synapse_data_folder = 'syn7872188'

# FOR TESTING ONLY
# synapse_data_folder = 'syn10163684'

synapse_data_folder_id = int(synapse_data_folder.replace('syn', ''))
storage_location_id = '9209'
nda_bucket_name = 'nda-bsmn'

In [2]:
s3 = boto3.resource("s3")
obj = s3.Object('kdaily-lambda-creds.sagebase.org', 'ndalogs_config.json')

config = json.loads(obj.get()['Body'].read())

ndaconfig = config['nda']

tokengenerator = nda_aws_token_generator.NDATokenGenerator()
mytoken = tokengenerator.generate_token(ndaconfig['username'],
                                        ndaconfig['password'])

session = boto3.Session(
    aws_access_key_id=mytoken.access_key,
    aws_secret_access_key=mytoken.secret_key,
    aws_session_token=mytoken.session
)

s3_nda = session.resource("s3")

# Get Samples

In [3]:
r = requests.get("https://ndar.nih.gov/api/guid/{}/data?short_name=genomics_sample03".format(REFERENCE_GUID), 
                 auth=requests.auth.HTTPBasicAuth(ndaconfig['username'], ndaconfig['password']),
                 headers={'Accept': 'application/json'})

guid_data = json.loads(r.text)

In [4]:
# Get data files from samples. There are currently up to two files per row.

tmp = [{col['name']: col['value'] for col in row['dataElement']} 
       for row in guid_data['age'][0]['dataStructureRow']]
    
samples = pandas.io.json.json_normalize(tmp)

colnames_lower = map(lambda x: x.lower(), samples.columns.tolist())
samples.columns = colnames_lower

# exclude some experiments
samples = samples[~samples.experiment_id.isin(EXCLUDE_EXPERIMENTS)]

samples['species'] = samples.organism.replace(['Homo Sapiens'], ['Human'])

samples1 = samples[['src_subject_id', 'experiment_id', 'subjectkey', 'sample_id_original', 'sample_id_biorepository',
                    'organism', 'species', 'sample_amount', 'sample_unit', 'biorepository',
                    'comments_misc', 'site', 'data_file1', 'data_file1_type']]

samples1.rename(columns={'data_file1': 'data_file', 'data_file1_type': 'fileFormat'}, inplace=True)

samples2 = samples[['src_subject_id', 'experiment_id', 'subjectkey', 'sample_id_original', 'sample_id_biorepository',
                    'organism', 'species', 'sample_amount', 'sample_unit', 'biorepository', 
                    'comments_misc', 'site', 'data_file2', 'data_file2_type']]

samples2.rename(columns={'data_file2': 'data_file', 'data_file2_type': 'fileFormat'}, inplace=True)

samples3 = pandas.concat([samples1, samples2], ignore_index=True)
samples3.filter(~samples3.data_file.isnull())
samples3['fileFormat'].replace(['BAM', 'FASTQ'], ['bam', 'fastq'], inplace=True)

# Remove initial slash to match what is in manifest file
samples3.data_file = samples3['data_file'].apply(lambda value: value[1:] if not pandas.isnull(value) else value)

# Remove stuff that isn't part of s3 path
samples3.data_file = map(lambda x: str(x).replace("![CDATA[", "").replace("]]>", ""), samples3.data_file.tolist())

samples3 = samples3[samples3.data_file != 'nan']

samples3.to_csv("./samples3.csv")

/usr/lib64/python2.7/site-packages/pandas/core/frame.py:2697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


# Get Subjects

In [5]:
r = requests.get("https://ndar.nih.gov/api/guid/{}/data?short_name=genomics_subject02".format(REFERENCE_GUID), 
                 auth=requests.auth.HTTPBasicAuth(ndaconfig['username'], ndaconfig['password']),
                 headers={'Accept': 'application/json'})

subject_guid_data = json.loads(r.text)

In [ ]:
tmp_subject = []
for row in subject_guid_data['age'][0]['dataStructureRow']:
    foo = {col['name']: col['value'] for col in row['dataElement']}
    tmp_subject.append(foo)
    
subjects = pandas.io.json.json_normalize(tmp_subject)
subjects = subjects[~subjects.GENOMICS_SUBJECT02_ID.isin(EXCLUDE_GENOMICS_SUBJECTS)]

colnames_lower = map(lambda x: x.lower(), subjects.columns.tolist())
subjects.columns = colnames_lower

subjects = subjects.assign(sex=subjects.gender.replace(['M', 'F'], ['male', 'female']),
                           subject_sample_id_original=subjects.sample_id_original,
                           subject_biorepository=subjects.biorepository)

subjects = subjects[['src_subject_id', 'subjectkey', 'gender', 'race', 'phenotype', 
                   'subject_sample_id_original', 'sample_description', 'subject_biorepository', 'sex']]

subjects = subjects.drop_duplicates()

subjects.to_csv("./subjects.csv")

# Get Tissues

In [7]:
r = requests.get("https://ndar.nih.gov/api/guid/{}/data?short_name=nichd_btb02".format(REFERENCE_GUID), 
                 auth=requests.auth.HTTPBasicAuth(ndaconfig['username'], ndaconfig['password']),
                 headers={'Accept': 'application/json'})

btb_guid_data = json.loads(r.text)

tmp_btb = []
for row in btb_guid_data['age'][0]['dataStructureRow']:
    foo = {col['name']: col['value'] for col in row['dataElement']}
    tmp_btb.append(foo)
    
btb = pandas.io.json.json_normalize(tmp_btb)

colnames_lower = map(lambda x: x.lower(), btb.columns.tolist())
btb.columns = colnames_lower

btb.drop('nichd_btb02_id', axis=1, inplace=True)
btb = btb.drop_duplicates()

btb.to_csv('./btb.csv')

# Get Manifests

Get list of `.manifest` files from the NDA-BSMN bucket. Read them in and concatenate them, under the assumption that the files listed in the manifest are in the same directory as the manifest file itself.

In [8]:
bucket = s3_nda.Bucket(nda_bucket_name)
manifests = [x for x in bucket.objects.all() if x.key.find('.manifest') >=0]

In [297]:
manifest = pandas.DataFrame()

for m in manifests:
    folder = os.path.split(m.key)[0]
    tmp = pandas.read_csv(m.get()['Body'], delimiter="\t", header=None)
    tmp.columns = ('filename', 'md5', 'size')
    tmp.filename = "s3://%s/%s/" % (nda_bucket_name, folder,) + tmp.filename.map(str)
    manifest = pandas.concat([manifest, tmp])

manifest.reset_index(drop=True, inplace=True)

manifest.to_csv('./manifest.csv')

Merge together the tissue file and the subjects file.

In [298]:
btb_subjects = btb.merge(subjects, how="left",
                         left_on=["src_subject_id", "subjectkey", "race", "gender"],
                         right_on=["src_subject_id", "subjectkey", "race", "gender"])

btb_subjects = btb_subjects.assign(sample_id_biorepository=btb_subjects.sample_id_original)

# Drop this as it will come back from the samples
btb_subjects.drop('sample_id_original', axis=1, inplace=True)

btb_subjects.to_csv('btb_subjects.csv')

Merge the tissue/subject with the samples to make a complete metadata table.

In [299]:
metadata = samples3.merge(btb_subjects, how="left",
                          left_on=["src_subject_id", "subjectkey", "sample_id_biorepository"],
                          right_on=["src_subject_id", "subjectkey", "sample_id_biorepository"])

metadata.index = metadata.data_file

metadata = metadata.drop_duplicates()

metadata = metadata[metadata_columns]

metadata.to_csv("./metadata.csv")

# Synapse

Using the concatenated manifests as the master list of files to store, create file handles and entities in Synapse.

Use the metadata table to get the appropriate tissue/subject/sample annotations to set on each File entity.

In [290]:
update = True

for n, x in manifest.iterrows():
    s3Key = x['filename'].replace("s3://%s/" % nda_bucket_name, "")
    s3FilePath = os.path.split(s3Key)[-1]
    contentSize = x['size']
    contentMd5 = x['md5']
    
    logger.debug("%s - %s" % (s3Key, s3FilePath))

    # Check if it exists in Synapse
    res = syn.restGET("/entity/md5/%s" % (contentMd5, ))['results']
    
    res = filter(lambda x: x['benefactorId'] == synapse_data_folder_id, res)
    
    if len(res) > 0:
        
        fhs = [syn.restGET("/entity/%(id)s/version/%(versionNumber)s/filehandles" % er) for er in res]    
        fileHandleObj = syn._getFileHandle(fhs[0]['list'][0]['id'])

        logger.info("%s already exists in Synapse (count = %s). Reusing fh %s" % (os.path.split(s3Key)[1], 
                                                                                  len(res), fileHandleObj))
            
    else:
        # print "Adding %s (%s)" % (contentMd5, x['filename'])

        
        contentType = content_type_dict.get(os.path.splitext(x['filename'])[-1],
                                            'application/octet-stream')
        
        fileHandle = {'concreteType': 'org.sagebionetworks.repo.model.file.S3FileHandle',
                      'fileName'    : s3FilePath,
                      'contentSize' : contentSize,
                      'contentType' : contentType,
                      'contentMd5' :  contentMd5,
                      'bucketName' : nda_bucket_name,
                      'key'        : s3Key,
                      'storageLocationId' : storage_location_id}

        fileHandleObj = syn.restPOST('/externalFileHandle/s3', 
                                     json.dumps(fileHandle), 
                                     endpoint=syn.fileHandleEndpoint)

    try:
        a = metadata.loc[x['filename']].to_dict()
        
        logger.debug("filename = %s, annotations = %s" % (x['filename'], a))

        f = synapseclient.File(parentId=synapse_data_folder, 
                               name=s3FilePath, 
                               dataFileHandleId = fileHandleObj['id'])
        f.annotations = a

        f = syn.store(f, forceVersion=False)
    except KeyError:
        logger.debug("Error getting metadata to annotation dictionary %s" % (x['filename'], ))


DEBUG:main:abyzova_1481389057198/Volumes/Promise Pegasus/data/s203958.TS_mosaicism/Project_Lt76/Sample_E_A1/A1_S1_L008_R1_001.fastq.gz - A1_S1_L008_R1_001.fastq.gz
INFO:main:A1_S1_L008_R1_001.fastq.gz already exists in Synapse (count = 6). Reusing fh {u'contentType': u'application/x-gzip', u'key': u'abyzova_1481389057198/Volumes/Promise Pegasus/data/s203958.TS_mosaicism/Project_Lt76/Sample_E_A1/A1_S1_L008_R1_001.fastq.gz', u'createdOn': u'2016-12-15T23:33:54.000Z', u'contentMd5': u'c5842fe9a698aa859aaaafa6fff291b6', u'concreteType': u'org.sagebionetworks.repo.model.file.S3FileHandle', u'etag': u'4169ead3-b753-4164-b3c1-5a5fb70c05b0', u'bucketName': u'nda-bsmn', u'createdBy': u'3323072', u'fileName': u'A1_S1_L008_R1_001.fastq.gz', u'storageLocationId': 9209, u'id': u'14178208', u'contentSize': 88507688}
INFO:main:Error getting metadata to annotation dictionary s3://nda-bsmn/abyzova_1481389057198/Volumes/Promise Pegasus/data/s203958.TS_mosaicism/Project_Lt76/Sample_E_A1/A1_S1_L008_R1_001

KeyboardInterrupt: 

In [289]:
manifest.loc[10].filename

u's3://nda-bsmn/abyzova_1481389057198/Volumes/Promise Pegasus/data/s203958.TS_mosaicism/Project_Lt76/Sample_E_A6/A6_S6_L008_R1_001.fastq.gz'

In [293]:
metadata

,src_subject_id,experiment_id,subjectkey,sample_id_original,sample_id_biorepository,subject_sample_id_original,biorepository,subject_biorepository,sample_description,species,site,sex,sample_amount,phenotype,comments_misc
data_file,,,,,,,,,,,,,,,
s3://nda-bsmn/abyzova_1481392262177/Volumes/Promise Pegasus/data/s203958.TS_mosaicism/Project_Lt76/Sample_E_C4/C4_S28_L008_R1_001.fastq.gz,5154,535,NDAR_INVRT663MBL,BSMN_REF_NeuN+_C4,BSMN_REF_NeuN+_C4,NaN,LIBD,NaN,NaN,Human,Yale/Mayo,NaN,11.71,NaN,Yale/Mayo;brain;dura;NeuN+;single nucleus;MDA;WGS;QC
s3://nda-bsmn/seancho_1492630428786/KKI_U01MH_JP-DuralFibroblast.zip,5154,533,NDAR_INVRT663MBL,5154_fibroblast,5154_fibroblast,5154,LIBD,LIBD,brain,Human,U01MH106892,male,379.43,normal,dural fibroblast for common experiment
s3://nda-bsmn/jmkidd_1491406438314/71938.lanes-merged.sort.RG.mkdups.realign.recal.bam,5154,641,NDAR_INVRT663MBL,71938,5154_fibroblast,5154,LIBD,LIBD,brain,NA,NaN,male,1,normal,NaN
s3://nda-bsmn/reneegeorge_1490833811716/BSM-Cells.final.bam,5154,643,NDAR_INVRT663MBL,BSM-Cells,5154_fibroblast,5154,LIBD,LIBD,brain,Human,NaN,male,50,normal,NaN
s3://nda-bsmn/reneegeorge_1490833811716/BSM-Br-3.final.bam,5154,643,NDAR_INVRT663MBL,BSM-Br-3,5154_brain,5154,LIBD,LIBD,brain,Human,NaN,male,50,normal,NaN
s3://nda-bsmn/reneegeorge_1490833811716/BSM-Br-2.final.bam,5154,643,NDAR_INVRT663MBL,BSM-Br-2,5154_brain,5154,LIBD,LIBD,brain,Human,NaN,male,50,normal,NaN
s3://nda-bsmn/reneegeorge_1490833811716/BSM-Br-1.final.bam,5154,643,NDAR_INVRT663MBL,BSM-Br-1,5154_brain,5154,LIBD,LIBD,brain,Human,NaN,male,50,normal,NaN
s3://nda-bsmn/abyzova_1481392262177/Volumes/Promise Pegasus/data/s203958.TS_mosaicism/Project_Lt76/Sample_E_C3/C3_S27_L008_R1_001.fastq.gz,5154,535,NDAR_INVRT663MBL,BSMN_REF_NeuN+_C3,BSMN_REF_NeuN+_C3,NaN,LIBD,NaN,NaN,Human,Yale/Mayo,NaN,12.16,NaN,Yale/Mayo;brain;dura;NeuN+;single nucleus;MDA;WGS;QC
s3://nda-bsmn/abyzova_1481392262177/Volumes/Promise Pegasus/data/s203958.TS_mosaicism/Project_Lt76/Sample_E_C2/C2_S26_L008_R1_001.fastq.gz,5154,535,NDAR_INVRT663MBL,BSMN_REF_NeuN+_C2,BSMN_REF_NeuN+_C2,NaN,LIBD,NaN,NaN,Human,Yale/Mayo,NaN,11.93,NaN,Yale/Mayo;brain;dura;NeuN+;single nucleus;MDA;WGS;QC
